In [1]:
import pandas as pd
import numpy as np
import warnings

In [2]:
atp_matches_2011 = pd.read_csv('atp_matches_2011.csv')
atp_matches_2012 = pd.read_csv('atp_matches_2012.csv')
atp_matches_2013 = pd.read_csv('atp_matches_2013.csv')
atp_matches_2014 = pd.read_csv('atp_matches_2014.csv')
atp_matches_2015 = pd.read_csv('atp_matches_2015.csv')
atp_matches_2016 = pd.read_csv('atp_matches_2016.csv')
atp_matches_2017 = pd.read_csv('atp_matches_2017.csv')
atp_matches_2018 = pd.read_csv('atp_matches_2018.csv')
atp_matches_2019 = pd.read_csv('atp_matches_2019.csv')
atp_matches_2020 = pd.read_csv('atp_matches_2020.csv')
atp_matches_2021 = pd.read_csv('atp_matches_2021.csv')
atp_matches_2022 = pd.read_csv('atp_matches_2022.csv')
atp_matches_2023 = pd.read_csv('atp_matches_2023.csv')
atp_matches_2024 = pd.read_csv('atp_matches_2024.csv')

In [3]:
df = pd.concat([atp_matches_2011, atp_matches_2012, atp_matches_2013, atp_matches_2014, atp_matches_2015, atp_matches_2016, atp_matches_2017, atp_matches_2018, atp_matches_2019, atp_matches_2020, atp_matches_2021, atp_matches_2022, atp_matches_2023, atp_matches_2024], ignore_index=True)

In [4]:
length = len(df)/2
y= []
for i in range(int(length)):
    y.append(0)
for i in range(int(length)):
    y.append(1)
df['y'] = pd.Series(y, index=df.index)

In [5]:
df1 = df[:int(length)]
df2 = df[int(length):]
L = list(df1)
for i in range(17, 27):
    L[i], L[i-10] = L[i-10], L[i]
df1= df1.reindex(columns = L)
cc = [df1, df2]
df = pd.concat(cc)

In [6]:
from sklearn.utils import shuffle
df = shuffle(df)


In [7]:
players = ['winner_id', 'winner_name', 'loser_id', 'loser_name']
to_drop = ['tourney_id', 'tourney_date', 'score', 'winner_seed', 'loser_seed']
cat_cols = ['tourney_name', 'surface', 'draw_size', 'tourney_level', 'winner_entry', 'winner_hand', 'winner_ioc', 'loser_entry', 'loser_hand', 'loser_ioc', 'best_of', 'round']
num_cols = ['match_num','winner_ht', 'winner_age','winner_rank','winner_rank_points','loser_ht','loser_age','loser_rank','loser_rank_points','minutes','w_ace','w_df', 'w_svpt','w_1stIn','w_1stWon','w_2ndWon','w_SvGms','w_bpSaved','w_bpFaced','l_ace','l_df','l_svpt','l_1stIn', 'l_1stWon','l_2ndWon','l_SvGms','l_bpSaved','l_bpFaced']

In [8]:
df = df.drop(to_drop, axis=1)
for i in cat_cols:
    df[i] = df[i].replace(np.nan, df[i].mode()[0])
for i in num_cols:
    df[i] = df[i].replace(np.nan, df[i].mean())

In [9]:
def convertCatToNum(dff):
    df_new = pd.get_dummies(dff, columns=cat_cols)
    return df_new
df = convertCatToNum(df)


In [10]:
def normalize(dff, col_name_list):
    result = dff.copy()
    for feature_name in col_name_list:
        max_value = dff[feature_name].max()
        min_value = dff[feature_name].min()
        result[feature_name] = (dff[feature_name] - min_value) / (max_value - min_value)
    return result
df = normalize(df, num_cols)

In [11]:
from sklearn.model_selection import train_test_split

Y = pd.DataFrame(df['y'])
df = df.drop(['y'], axis=1)
X = df 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [55]:
df.shape

(37882, 1308)

In [14]:
X_train = X_train.drop(['winner_id'], axis=1)
X_train = X_train.drop(['winner_name'], axis=1)
X_train = X_train.drop(['loser_id'], axis=1)
X_train = X_train.drop(['loser_name'], axis=1)
test_names = []
for index, row in X_test.iterrows():
    v = [row['winner_id'], row['loser_id']]
    test_names.append(v)
X_test = X_test.drop(['winner_id'], axis=1)
X_test = X_test.drop(['winner_name'], axis=1)
X_test = X_test.drop(['loser_id'], axis=1)
X_test = X_test.drop(['loser_name'], axis=1)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
model = LogisticRegression()

parametric_grid = {
    'C' : np.logspace(0.0, 4.0, num = 10),
    'penalty' : ['l1', 'l2'], 
    'solver' :['liblinear', 'saga'],
}
search = RandomizedSearchCV(model, parametric_grid, n_iter=5, cv=10, scoring='accuracy', n_jobs=-1, random_state=1)

result = search.fit(X_train, y_train)
print('Best Score: %s' %result.best_score_)
print('Best Hyperparameters: %s' %result.best_params_)

C:\Users\Ananya\PycharmProjects\Tennis Project\.venv\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Score: 0.9034481534593578
Best Hyperparameters: {'solver': 'saga', 'penalty': 'l1', 'C': 166.81005372000593}


C:\Users\Ananya\PycharmProjects\Tennis Project\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [16]:
from sklearn.metrics import classification_report
best_random = result.best_estimator_
y_pred_test = best_random.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.95      0.84      0.90      3798
           1       0.86      0.96      0.91      3779

    accuracy                           0.90      7577
   macro avg       0.91      0.90      0.90      7577
weighted avg       0.91      0.90      0.90      7577



In [31]:
model.fit(X, Y)


LogisticRegression()

In [21]:
warnings.filterwarnings('ignore')

In [40]:

model.predict(test_names)

ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [ ]:
X_train = X_train.astype(['winner_id'], axis=1)

In [30]:
X = X.drop(['winner_name'], axis=1)
X = X.drop(['loser_name'], axis=1)